<a href="https://colab.research.google.com/github/canamac/zeka-can/blob/main/1_G%C3%9CNL%C3%9CK_PER%C4%B0YOD_15_DAK_MUM_KAPANI%C5%9EI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from rich.console import Console
from rich.table import Table
from datetime import datetime, timedelta

def get_data(symbols, period='1d', interval='15m'):
    data = {}
    for symbol in symbols:
        try:
            ticker = yf.Ticker(symbol)
            df = ticker.history(period=period, interval=interval)
            if not df.empty and len(df) > 34:
                data[symbol] = df
        except Exception as e:
            print(f"Hata: {symbol} - {str(e)}")
    return data

def calculate_indicators(df, params):
    try:
        # RSI Hesaplama
        delta = df['Close'].diff()
        gain = delta.where(delta > 0, 0)
        loss = -delta.where(delta < 0, 0)

        avg_gain = gain.ewm(alpha=1/params['rsi_length'], adjust=False).mean()
        avg_loss = loss.ewm(alpha=1/params['rsi_length'], adjust=False).mean()
        rs = avg_gain / avg_loss
        rsi = 100 - (100 / (1 + rs))

        # MFI Hesaplama
        typical_price = (df['High'] + df['Low'] + df['Close']) / 3
        raw_money_flow = typical_price * df['Volume']

        positive_flow = raw_money_flow.where(typical_price > typical_price.shift(1), 0)
        negative_flow = raw_money_flow.where(typical_price < typical_price.shift(1), 0)

        money_ratio = (positive_flow.rolling(params['mfi_length']).sum() /
                      negative_flow.rolling(params['mfi_length']).sum())
        mfi = 100 - (100 / (1 + money_ratio))

        # EMA Delta
        ema_short = df['Close'].ewm(span=params['ema_short'], adjust=False).mean()
        ema_long = df['Close'].ewm(span=params['ema_long'], adjust=False).mean()
        delta_ema = ema_short - ema_long

        # RSI Değişim Sinyalleri
        log_close = np.log(df['Close'])
        rsi_log = 100 - (100 / (1 + (log_close.diff().clip(lower=0).ewm(14).mean() /
                                    log_close.diff().clip(upper=0).abs().ewm(14).mean())))
        rsi_change = rsi_log.diff()

        return {
            'Price': df['Close'].iloc[-1],
            'RSI': rsi.iloc[-1],
            'MFI': mfi.iloc[-1],
            'Delta EMA': delta_ema.iloc[-1],
            'RSI Change': rsi_change.iloc[-1],
            'C20L': rsi_change.iloc[-1] > 20,
            'C10L': rsi_change.iloc[-1] > 10,
            'C20S': rsi_change.iloc[-1] < -20,
            'C10S': rsi_change.iloc[-1] < -10
        }
    except Exception as e:
        print(f"Hesaplama hatası: {e}")
        return None

def create_monitoring_table(data):
    console = Console()
    table = Table(title="🚨 50 Coin Realtime Signals", show_header=True, header_style="bold cyan")

    columns = [
        ("No", 4), ("Symbol", 12), ("Price", 12),
        ("RSI", 8), ("MFI", 8), ("Delta EMA", 10),
        ("RSI Change", 12), ("C20L", 6), ("C10L", 6),
        ("C20S", 6), ("C10S", 6)
    ]

    for col, width in columns:
        table.add_column(col, width=width, justify="right")

    for idx, (symbol, metrics) in enumerate(data.items(), 1):
        if metrics:
            row = [
                str(idx),
                symbol,
                f"{metrics['Price']:,.2f}",
                f"{metrics['RSI']:.1f}",
                f"{metrics['MFI']:.1f}" if not np.isnan(metrics['MFI']) else "N/A",
                f"{metrics['Delta EMA']:+.4f}",
                f"{metrics['RSI Change']:+.2f}",
                "[bold green]✓[/bold green]" if metrics['C20L'] else "[gray]-[/gray]",
                "[green]✓[/green]" if metrics['C10L'] else "[gray]-[/gray]",
                "[bold red]✓[/bold red]" if metrics['C20S'] else "[gray]-[/gray]",
                "[red]✓[/red]" if metrics['C10S'] else "[gray]-[/gray]"
            ]
            table.add_row(*row)

    console.print(table)

def main():
    params = {
        'rsi_length': 14,
        'mfi_length': 14,
        'ema_short': 10,
        'ema_long': 34
    }

    symbols = [
        'BTC-USD', 'ETH-USD', 'BNB-USD', 'XRP-USD', 'SOL-USD',
        'ADA-USD', 'DOGE-USD', 'MATIC-USD', 'DOT-USD', 'LTC-USD',
        'AVAX-USD', 'LINK-USD', 'ATOM-USD', 'UNI-USD', 'XLM-USD',
        'ALGO-USD', 'TRX-USD', 'ETC-USD', 'FTT-USD', 'XMR-USD',
        'APE-USD', 'SAND-USD', 'MANA-USD', 'NEAR-USD', 'EGLD-USD',
        'FTM-USD', 'GRT-USD', 'AAVE-USD', 'SNX-USD', 'CRV-USD',
        'MKR-USD', 'COMP-USD', 'ZEC-USD', 'DASH-USD', 'ENJ-USD',
        'IOTA-USD', 'XTZ-USD', 'CHZ-USD', 'KSM-USD', 'AR-USD',
        'RUNE-USD', 'QTUM-USD', 'ZIL-USD', 'BAT-USD', 'ONE-USD',
        'ONT-USD', 'ICX-USD', 'SC-USD', 'ANKR-USD', 'RVN-USD'
    ]

    print("🔁 Veriler güncelleniyor...")
    data = get_data(symbols)

    processed = {}
    for sym, df in data.items():
        res = calculate_indicators(df, params)
        if res:
            processed[sym] = res

    create_monitoring_table(processed)

if __name__ == "__main__":
    main()

🔁 Veriler güncelleniyor...


ERROR:yfinance:$FTM-USD: possibly delisted; no price data found  (period=1d)
ERROR:yfinance:$GRT-USD: possibly delisted; no price data found  (period=1d)
ERROR:yfinance:$COMP-USD: possibly delisted; no price data found  (period=1d)


                                            🚨 50 Coin Realtime Signals                                            
┏━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃  No ┃      Symbol ┃       Price ┃     RSI ┃     MFI ┃ Delta EMA ┃  RSI Change ┃   C20L ┃  C10L ┃   C20S ┃  C10S ┃
┡━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│   1 │     BTC-USD │  101,575.26 │    20.7 │    11.7 │ -1242.05… │       -0.54 │      - │     - │      - │     - │
│   2 │     ETH-USD │    3,294.04 │    34.4 │    41.2 │  -17.5628 │       -1.19 │      - │     - │      - │     - │
│   3 │     BNB-USD │      675.00 │    27.4 │    40.9 │   -2.8085 │       -1.14 │      - │     - │      - │     - │
│   4 │     XRP-USD │        3.02 │    30.4 │    17.9 │   -0.0288 │       -0.72 │      - │     - │      - │     - │
│   5 │     SOL-USD │      229.21 │    26.3 │    17.2 │   -3.4873 │       -0.31 │      - │     - │      - │     - │
│   6 │     ADA-USD │        0.94 │    32.0 │    24.2 │   -0.0102 │       -1.08 │      - │     - │      - │     - │
│   7 │    DOGE-USD │        0.33 │    34.7 │    21.2 │   -0.0033 │       -0.76 │      - │     - │      - │     - │
│   8 │   MATIC-USD │        0.40 │    26.6 │    21.8 │   -0.0042 │       -0.51 │      - │     - │      - │     - │
│   9 │     DOT-USD │        6.21 │    32.1 │    27.5 │   -0.0473 │       -1.42 │      - │     - │      - │     - │
│  10 │     LTC-USD │      127.35 │    43.1 │    33.5 │   -1.1265 │       -1.62 │      - │     - │      - │     - │
│  11 │    AVAX-USD │       34.31 │    30.5 │    29.0 │   -0.3705 │       -0.41 │      - │     - │      - │     - │
│  12 │    LINK-USD │       24.66 │    30.7 │    25.9 │   -0.3297 │       -0.42 │      - │     - │      - │     - │
│  13 │    ATOM-USD │        6.19 │    33.2 │    33.2 │   -0.0421 │       -1.09 │      - │     - │      - │     - │
│  14 │     UNI-USD │        0.00 │    21.9 │     N/A │   -0.0000 │       -1.02 │      - │     - │      - │     - │
│  15 │     XLM-USD │        0.41 │    42.3 │    27.2 │   -0.0039 │       -1.11 │      - │     - │      - │     - │
│  16 │    ALGO-USD │        0.38 │    25.8 │    26.5 │   -0.0064 │       -0.69 │      - │     - │      - │     - │
│  17 │     TRX-USD │        0.25 │    32.5 │    12.8 │   -0.0010 │       -0.79 │      - │     - │      - │     - │
│  18 │     ETC-USD │       26.82 │    37.0 │    40.5 │   -0.0548 │       -1.52 │      - │     - │      - │     - │
│  19 │     FTT-USD │        2.18 │    36.7 │    39.7 │   -0.0147 │       -0.03 │      - │     - │      - │     - │
│  20 │     XMR-USD │      232.59 │    34.0 │    24.1 │   -0.7723 │       -0.28 │      - │     - │      - │     - │
│  21 │     APE-USD │        0.92 │    67.4 │    76.1 │   +0.0476 │       +0.00 │      - │     - │      - │     - │
│  22 │    SAND-USD │        0.53 │    32.9 │    32.5 │   -0.0052 │       -0.94 │      - │     - │      - │     - │
│  23 │    MANA-USD │        0.45 │    33.3 │    30.9 │   -0.0041 │       -0.98 │      - │     - │      - │     - │
│  24 │    NEAR-USD │        4.61 │    32.0 │    28.2 │   -0.0457 │       -1.40 │      - │     - │      - │     - │
│  25 │    EGLD-USD │       29.68 │    31.0 │    16.2 │   -0.3116 │       -0.31 │      - │     - │      - │     - │
│  26 │    AAVE-USD │      333.62 │    47.4 │    50.2 │   +1.4480 │       -0.92 │      - │     - │      - │     - │
│  27 │     SNX-USD │        1.59 │    32.8 │    47.6 │   -0.0181 │       -0.50 │      - │     - │      - │     - │
│  28 │     CRV-USD │        0.77 │    34.5 │    29.8 │   -0.0121 │       -0.48 │      - │     - │      - │     - │
│  29 │     MKR-USD │    1,141.87 │    32.4 │    17.8 │   -7.2475 │       -0.15 │      - │     - │      - │     - │
│  30 │     ZEC-USD │       42.96 │    29.6 │    27.9 │   -0.5652 │       -1.16 │      - │     - │      - │     - │
│  31 │    DASH-USD │       33.37 │    31.7 │    93.5 │  